# ***Prepare***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import sys
sys.path.append('/content/drive/MyDrive/python_env/lib/python3.10/site-packages')

project_path = "/content/drive/MyDrive/AI_MRI_Project"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

# ***Load Model***

In [ ]:
# Import necessary libraries
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# ======================= CONFIGURATION =======================
# The original configuration for BitsAndBytesConfig was commented out.
# BitsAndBytesConfig is useful for loading models with 4-bit quantization for reduced memory consumption.
# This configuration is optimal for balancing performance and precision when using large models like LLaMA.
#
# Example configuration for reference:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,               # ✅ Load the model in 4-bit precision to reduce memory consumption
#     bnb_4bit_compute_dtype=torch.float16, # ✅ Use FP16 for efficient computation with minimal precision loss
#     bnb_4bit_use_double_quant=True,  # ✅ Apply second-layer quantization to improve model performance
#     bnb_4bit_quant_type="nf4",       # ✅ Use NF4 (Normalized Float 4) for better accuracy in 4-bit models
# )

# ======================= BASE MODEL LOADING =======================
# Define the base model name using Meta's LLaMA 2 7B chat variant.
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the base LLaMA model for causal language modeling (causal LM).
# Causal LM predicts the next token based on the previous tokens.
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the tokenizer for the base model.
# Tokenizer is responsible for text tokenization and detokenization processes.
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# ======================= LoRA FINE-TUNED MODEL LOADING =======================
# Path to the fine-tuned model trained with PEFT (Parameter Efficient Fine-Tuning)
fine_tuned_model_path = '/content/drive/MyDrive/AI_MRI_Project/fine_tuned_models'

# Load the fine-tuned model using PEFT, which efficiently updates fewer model parameters for faster training and improved memory efficiency.
fine_tuned_model = PeftModel.from_pretrained(base_model, fine_tuned_model_path)

# Use the same tokenizer for the fine-tuned model to ensure consistency in text encoding/decoding.
fine_tuned_tokenizer = base_tokenizer

# ======================= TEXT GENERATION PIPELINES =======================
# Pipeline for text generation using the base LLaMA model
# `torch_dtype=torch.float16` ensures reduced memory usage and faster computations using FP16 precision.
# `device_map="auto"` automatically maps the model to available GPUs or CPUs for optimized performance.

# Pipeline for text generation using the fine-tuned LLaMA model
# Fine-tuned models often demonstrate improved accuracy for domain-specific tasks.
fine_tuned_generator = pipeline(
    "text-generation",  # Defines the task as text generation
    model=fine_tuned_model,  # Fine-tuned PEFT model
    tokenizer=fine_tuned_tokenizer,  # Consistent tokenizer for the fine-tuned model
    torch_dtype=torch.float16,  # FP16 for better speed and memory efficiency
    device_map="auto"  # Automatically allocate computing resources for efficient model inference
)







/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'Gra

# ***Findings***

In [ ]:
import re

def clean_mri_findings_output(raw_output):
    """
    Cleans LLaMA model output and extracts relevant MRI Findings information.

    This function is designed to process raw output generated by the LLaMA model,
    identifying and extracting content specifically labeled as "MRI Findings."
    It relies on a structured format that includes '**BEGIN OUTPUT**' as a key marker.

    Parameters:
    - raw_output (str): The raw text output generated by the LLaMA model.

    Returns:
    - str: The extracted MRI Findings content.
           If no valid content is detected, an error message is returned.
    """

    # ======================= Regular Expression Pattern =======================
    # The regex pattern is designed to:
    # 🔹 Locate content starting immediately after '**BEGIN OUTPUT**' (ignoring surrounding text)
    # 🔹 Capture all content until the next double newline `\n\n` or the end of the text `$`.
    # 🔹 `(?<=...)` is a **positive lookbehind assertion** — it ensures the match starts
    #    immediately after the '**BEGIN OUTPUT**' marker but excludes the marker itself.
    # 🔹 `(?=\n\n|$)` is a **positive lookahead assertion** — it stops the match before
    #    encountering either a double newline or the end of the text.

    pattern = r"(?<=\*\*BEGIN OUTPUT\*\*\n)(.*?)(?=\n\n|$)"

    # ======================= Perform Regex Search =======================
    # `re.search()` is used because:
    # 🔹 It efficiently finds the first match within the text.
    # 🔹 `re.DOTALL` is used to ensure `.` matches newline characters,
    #    allowing multi-line MRI findings content to be captured.
    match = re.search(pattern, raw_output, re.DOTALL)

    # ======================= Extraction & Return =======================
    # ✅ If a match is found, return the cleaned content (trimmed for clarity)
    if match:
        return match.group(0).strip()

    # ❌ If no match is found, return an error message
    # This message is useful for debugging prompts, model outputs, or formatting issues.
    else:
        return "❌ MRI Findings not found. Please check the prompt format or model output."



In [ ]:
def generate_mri_findings(mri_description, llama_model):
    """
    Generates MRI Findings based on a provided MRI imaging description using LLaMA model.

    This function employs a One-Shot Prompting technique, where a structured example is given
    to guide the model toward generating accurate and standardized MRI findings.

    Parameters:
    - mri_description (str): The descriptive text explaining MRI imaging observations.
    - llama_model (Pipeline): The LLaMA model (or other compatible model) for text generation.

    Returns:
    - str: The generated MRI Findings output based on the given description.
    """

    # ======================= Step 1: Prompt Engineering =======================
    # ✅ Prompt Design Strategy: One-Shot Prompting
    # One-shot prompting provides the model with:
    #   1. Clear task instructions
    #   2. An example output in the desired format
    #   3. The actual MRI imaging description for the model to interpret
    # This structured format helps the model produce coherent and standardized text output.

    prompt = f"""
    Generate the 'Findings' section of a standard MRI report based on the following MRI imaging description.

    **Imaging Description**:
    MRI scan shows a well-defined extra-axial mass in the left parietal region, measuring 3.0 cm.
    The lesion is isointense on T1-weighted imaging and hyperintense on T2-weighted imaging.
    The mass exhibits a clear boundary with localized growth extending along the dura.
    There is no evidence of surrounding edema or significant mass effect.

    **Example Output (Follow this format)**:
    - **Tumor Classification**: Meningioma
    - **Location**: Left parietal region
    - **Size/Extent**: 3.0 cm
    - **Type**: Isointense on T1-weighted imaging, hyperintense on T2-weighted imaging
    - **Distribution Pattern**: Extra-axial lesion with clear boundaries, localized growth pattern
    - **Mass Effect**: No significant mass effect or surrounding edema

    Now, based on the following MRI imaging description, generate the corresponding 'Findings' section:

    **Imaging Description**:
    {mri_description}

    **BEGIN OUTPUT**
    """

    # ======================= Step 2: Model Generation =======================
    # ✅ LLaMA Model Inference
    # `max_length=2048` is set to accommodate detailed MRI findings descriptions.
    # - Large medical descriptions and multi-feature outputs may require extended token limits.
    # `temperature=0.3` is selected to:
    # - Reduce randomness in the output for more precise and factual text generation.
    # - Lower temperature values encourage more deterministic outputs, which are ideal for medical content.
    response = llama_model(prompt, max_length=2048, temperature=0.3)[0]["generated_text"]

    # ✅ Return the generated MRI findings text
    return response



In [ ]:
mri_description = "MRI scan reveals a large hyperintense mass in the right frontal lobe, measuring approximately 4.5 cm. The lesion demonstrates irregular margins with heterogeneous enhancement following contrast administration. Significant vasogenic edema is present, causing a midline shift of approximately 5 mm. There is compression of the right lateral ventricle with partial effacement of the sulci."


In [ ]:
mri_description = "Symmetrical enhancement in the brainstem, particularly within the pons region, indicating potential inflammatory, neoplastic, or vascular pathology. A hyperintense lesion with irregular enhancement in the posterior fossa, notably in the cerebellum, suggesting possible secondary involvement or extension.No significant midline shift, ventricular compression, or sulcal effacement observed. The surrounding brain parenchyma appears structurally preserved, with no signs of acute hemorrhage or ischemia."

In [ ]:
# Generate MRI Findings using LLaMA
raw_mri_findings = generate_mri_findings(mri_description, fine_tuned_generator)

# Clean the model's output to extract MRI findings only
mri_findings = clean_mri_findings_output(raw_mri_findings)

# Display Results
print("📑 MRI Findings:\n", mri_findings)

📑 MRI Findings:
 **Tumor Classification**: None identified
    **Location**: Brainstem and posterior fossa
    **Size/Extent**: N/A
    **Type**: Symmetrical enhancement in the brainstem, hyperintense lesion in the posterior fossa
    **Distribution Pattern**: Brainstem and posterior fossa involvement with irregular enhancement
    **Mass Effect**: No significant mass effect or surrounding edema


# ***Risk Assessment***

In [ ]:
def assess_mri_risk(mri_findings, llama_model):
    """
    Assess the MRI risk level based on MRI findings using the LLaMA model.

    This function utilizes a structured prompt that guides the model to:
    - Follow a predefined scoring system for risk assessment.
    - Provide both a numerical score and a detailed justification.
    - Clearly explain how each MRI characteristic contributes to the final risk score.

    Parameters:
    - mri_findings (str): The detailed MRI findings containing tumor classification, size, location, etc.
    - llama_model (Pipeline): The LLaMA model (or any text generation model) used for inference.

    Returns:
    - str: The generated risk assessment section with detailed scoring and justifications.
    """

    # ======================= Step 1: Prompt Engineering =======================

    # ✅ Prompt Design Strategy: Structured Prompting
    # The prompt uses a clear structure with:
    # 1️⃣ Example Output: Demonstrates the expected structure and format.
    # 2️⃣ Scoring Guidelines: Guides the model on how to assign risk scores based on MRI characteristics.
    # 3️⃣ MRI Findings: Injects the provided MRI findings for customized risk assessment.

    prompt = f"""
    Generate the 'Risk Assessment' section of a standard MRI report based on the following MRI findings.

    **Example Output (Follow this format)**:
    - **Risk Score**: [Total score out of 100]
    - **Risk Level**: [Low/Medium/High]
    - **Justification**:
      - **Tumor Classification**: [Explain contribution to risk score]
      - **Location**: [Explain contribution to risk score]
      - **Size/Extent**: [Explain contribution to risk score]
      - **Distribution Pattern**: [Explain contribution to risk score]
      - **Mass Effect**: [Explain contribution to risk score]

    # ======================= Step 2: Scoring Guidelines =======================
    # ✅ Clear and Weighted Scoring Rules
    # Each MRI characteristic is assigned a percentage weight (e.g., Tumor Classification = 20%).
    # The model uses these guidelines to calculate a final risk score (out of 100).

    **Scoring Guidelines** (Use this to calculate the score):

    1️⃣ **Tumor Classification (20%)**:
      - 🔴 **High Risk (Score 18-20)**: Aggressive tumors like Glioblastoma (GBM) or high-grade gliomas
      - 🟡 **Medium Risk (Score 8-17)**: Intermediate-grade tumors like anaplastic astrocytoma
      - 🟢 **Low Risk (Score 0-7)**: Low-grade gliomas, benign tumors

    2️⃣ **Location (15%)**:
      - 🔴 **High Risk (Score 13-15)**: Located in critical regions (brainstem, thalamus, basal ganglia)
      - 🟡 **Medium Risk (Score 6-12)**: Near functional cortex but non-invasive
      - 🟢 **Low Risk (Score 0-5)**: Located in non-critical peripheral regions

    3️⃣ **Size/Extent (30%)**:
      - 🔴 **High Risk (Score 28-30)**: > 4 cm
      - 🟡 **Medium Risk (Score 15-27)**: 2 - 4 cm
      - 🟢 **Low Risk (Score 0-14)**: < 2 cm

    4️⃣ **Distribution Pattern (15%)**:
      - 🔴 **High Risk (Score 13-15)**: Irregular margins with signs of tissue infiltration
      - 🟡 **Medium Risk (Score 6-12)**: Localized lesion with mild expansion
      - 🟢 **Low Risk (Score 0-5)**: Well-defined borders, no tissue infiltration

    5️⃣ **Mass Effect (20%)**:
      - 🔴 **High Risk (Score 18-20)**: Midline shift ≥ 5 mm, severe vasogenic edema
      - 🟡 **Medium Risk (Score 8-17)**: Mild mass effect, localized sulci effacement
      - 🟢 **Low Risk (Score 0-7)**: No significant mass effect

    # ======================= Step 3: Insert MRI Findings =======================
    # ✅ MRI findings dynamically inserted
    # The prompt appends the provided `mri_findings` for the model to analyze.

    Now, based on the following MRI findings, generate the corresponding 'Risk Assessment' section:

    **MRI Findings**:
    {mri_findings}

    **BEGIN OUTPUT**
    """

    # ======================= Step 4: Model Inference =======================

    # ✅ LLaMA Model Inference
    # `max_length=2048` ensures sufficient token space for complex reports.
    # `temperature=0.3` reduces randomness, promoting clear and factual outputs.
    response = llama_model(prompt, max_length=2048, temperature=0.3)[0]["generated_text"]

    # ======================= Step 5: Return the Response =======================
    return response



In [ ]:
import re

def extract_output_section(raw_text):
    """
    Extracts the content between '**BEGIN OUTPUT**' and '**END OUTPUT**'.

    This function uses a regular expression (regex) pattern to locate and extract
    the desired text section. The expected format is:

    **BEGIN OUTPUT**
    [Desired Content]
    **END OUTPUT**

    Parameters:
    - raw_text (str): The raw text containing the desired content.

    Returns:
    - str: The extracted text content. If no valid content is found,
           an appropriate error message is returned.
    """

    # ======================= Step 1: Regular Expression Design =======================
    # ✅ Regex Pattern: r"\*\*BEGIN OUTPUT\*\*(.*?)\*\*END OUTPUT\*\*"
    # Explanation:
    # 🔹 `\*\*` — Escapes the `**` marker (required because `*` is a special character in regex).
    # 🔹 `BEGIN OUTPUT` and `END OUTPUT` — Keywords marking the boundaries of the desired text.
    # 🔹 `(.*?)` — Captures text **non-greedily** between the markers.
    #     - `?` ensures the search stops at the **first** `**END OUTPUT**` it encounters.
    # 🔹 `re.DOTALL` — Enables `.` to match newline characters, ensuring multi-line content is captured.

    extracted_text = re.search(r"\*\*BEGIN OUTPUT\*\*(.*?)\*\*END OUTPUT\*\*",
                               raw_text,
                               re.DOTALL)

    # ======================= Step 2: Extract & Return Results =======================
    # ✅ Check if a match was found
    if extracted_text:
        # ✅ `group(1)` extracts the captured content inside the `(.*?)`
        # `.strip()` is used to remove leading/trailing spaces for cleaner output.
        return extracted_text.group(1).strip()

    # ❌ If no valid content is found, return an error message
    else:
        return "❗️ Unable to locate content. Please check the original text format."


In [ ]:
# Step 1: Generate Raw Risk Assessment
# -------------------------------------
# ✅ Generates raw MRI risk assessment text using the fine-tuned generator model
# `mri_description` should contain the MRI report's detailed findings
raw_risk_assessment = assess_mri_risk(mri_description, fine_tuned_generator)

# Step 2: Extract Cleaned Risk Assessment
# ---------------------------------------
# ✅ Extracts only the relevant content from the generated text
# The `extract_output_section()` function isolates content between "**BEGIN OUTPUT**" and "**END OUTPUT**"
risk_assessment = extract_output_section(raw_risk_assessment)

# Step 3: Display the Final Risk Assessment
# -----------------------------------------
# ✅ Prints the cleaned MRI risk assessment for clear presentation
print('Risk Assessment')
print(risk_assessment)

Risk Assessment
   The risk of malignancy is estimated to be [High (70/100)]
   The contribution to the risk score is as follows:
        - Tumor Classification: Contributes 20% to the total risk score. The glioma classification is consistent with a medium to
                                high-grade tumor, which accounts for [80/100] of the total risk score.
        - Location: Contributes 15% to the total risk score. The right frontal lobe location is a medium-risk factor, accounting
                       for [12/100] of the total risk score.
        - Size/Extent: Contributes 30% to the total risk score. The tumor size of 4.5 cm is a medium-risk factor, accounting for
                       [45/100] of the total risk score.
        - Distribution Pattern: Contributes 15% to the total risk score. The irregular margins and significant midline shift with
                       vasogenic edema are high-risk features, accounting for [22/100] of the total risk score.
.       - Mass Effe

# ***Treatment Plan***

In [ ]:
def generate_treatment_plan(mri_findings, risk_level, llama_model):
    """
    Generates a personalized treatment plan based on MRI findings and risk assessment using the LLaMA model.

    This function employs One-Shot Prompting, providing the model with:
    - A clear task description
    - An example input with detailed MRI findings and risk assessment
    - A well-structured example output to guide the model's response

    Parameters:
    - mri_findings (str): MRI diagnostic details including tumor type, location, size, etc.
    - risk_level (str): Risk assessment summary (e.g., Low, Medium, or High risk).
    - llama_model (Pipeline): The LLaMA model (or compatible generator) used for text generation.

    Returns:
    - str: A personalized treatment recommendation based on the provided MRI data.
    """

    # ======================= Step 1: Prompt Engineering =======================
    # ✅ Prompt Design Strategy: One-Shot Prompting
    # This structured prompt effectively guides the LLaMA model to generate reliable outputs.
    # Key components:
    #   1️⃣ Clear Instructions
    #   2️⃣ Example Input (Demonstrating expected data structure)
    #   3️⃣ Example Output (Demonstrating desired treatment plan format)
    #   4️⃣ Dynamic Insertion of MRI Findings and Risk Level for personalized recommendations

    prompt = f"""
    Generate a personalized 'Treatment Recommendations' section of an MRI diagnostic report.
    The treatment plan should consider:
    - MRI Findings
    - Risk Assessment
    - Predicted Diseases

    **Example Input:**
    **MRI Findings**:
    - **Tumor Classification**: Meningioma
    - **Location**: Left parietal region
    - **Size/Extent**: 3.0 cm
    - **Type**: Isointense on T1-weighted imaging, hyperintense on T2-weighted imaging
    - **Distribution Pattern**: Extra-axial lesion with clear boundaries, localized growth pattern
    - **Mass Effect**: No significant mass effect or surrounding edema

    **Risk Assessment:** Medium Risk (Score: 55/100)

    **Predicted Disease(s)**: Meningioma (85% confidence)

    **Example Output (Follow this format):**
    - **Imaging Recommendations**: Perform contrast-enhanced MRI to assess vascular involvement.
    - **Follow-up Schedule**: MRI follow-up in 3 months to monitor lesion stability.
    - **Treatment Suggestions**:
        - Surgery may be considered if the lesion shows rapid growth.
        - Radiation therapy may be indicated for residual tumor or recurrence.
        - Regular monitoring for neurological symptoms is advised.

    Now, based on the following information, generate the corresponding 'Treatment Recommendations' section:

    **MRI Findings**:
    {mri_findings}

    **Risk Assessment:** {risk_level}

    **BEGIN OUTPUT**
    """

    # ======================= Step 2: Model Inference =======================
    # ✅ LLaMA Model Inference
    # The following parameters ensure the model generates focused, structured, and accurate content:
    # 🔹 `max_length=2048` — Ensures sufficient space for complex treatment recommendations.
    # 🔹 `temperature=0.3` — Low temperature favors deterministic, factual content over creative text.
    #    - Ideal for generating precise medical content rather than open-ended language.

    response = llama_model(prompt, max_length=2048, temperature=0.3)[0]["generated_text"]

    # ======================= Step 3: Return the Response =======================
    return response



In [4]:
# Step 1: Generate Treatment Plan
# --------------------------------
# ✅ Uses the `generate_treatment_plan()` function to generate a personalized treatment plan.
# `mri_findings` contains the MRI diagnostic details.
# `risk_assessment` is the evaluated risk score (e.g., Low, Medium, or High Risk).
# `fine_tuned_generator` is the model generating the text output.
treatment_plan = generate_treatment_plan(mri_findings, risk_assessment, fine_tuned_generator)

# Step 2: Extract Cleaned Treatment Plan
# --------------------------------------
# ✅ The raw treatment plan may include extra content.
# The `extract_output_section()` function isolates only the meaningful part (from **BEGIN OUTPUT** to **END OUTPUT**).
treatment_plan = extract_output_section(treatment_plan)

# Step 3: Display Treatment Plan
# ------------------------------
# ✅ Print the clean and structured treatment plan.
print(treatment_plan)


Based on the MRI findings and risk assessment, the following treatment options are considered:

- **Surgical Resection**: Surgical resection may be considered for this patient, given the high risk
  of malignancy and significant midline shift. The tumor's location in the right frontal lobe and
  its irregular margins may make it challenging to completely resect the tumor, but it is still
  the primary treatment option.

- **Radiation Therapy**: Radiation therapy may be considered as an adjunct to surgical resection
  or as a standalone treatment for any remaining tumor after surgery. The patient's age and overall
  health status may affect the radiation therapy plan.

- **Conservative Management**: Conservative management with close follow-up and serial MRI may be
  considered for this patient, given the high risk of malignancy and significant midline shift.
  This approach may be more appropriate for patients who are not candidates for surgical resection
  or radiation therapy.

- **C

# ***Generate Reports***

In [5]:
def generate_mri_diagnostic_report(mri_description, fine_tuned_generator):
    """
    Generates a complete MRI diagnostic report that includes:
    1. MRI Findings
    2. Risk Assessment
    3. Treatment Recommendations

    Parameters:
    - mri_description (str): The detailed description of MRI imaging observations.
    - fine_tuned_generator (Pipeline): The fine-tuned LLaMA model used for text generation.

    Returns:
    - str: A formatted diagnostic report combining all sections.
    """

    # Step 1: Generate MRI Findings
    # ------------------------------
    # ✅ Generate MRI Findings section using the fine-tuned model
    raw_mri_findings = generate_mri_findings(mri_description, fine_tuned_generator)
    mri_findings = clean_mri_findings_output(raw_mri_findings)

    # Step 2: Generate Risk Assessment
    # ---------------------------------
    # ✅ Generate the risk assessment using the MRI findings
    raw_risk_assessment = assess_mri_risk(mri_findings, fine_tuned_generator)
    risk_assessment = extract_output_section(raw_risk_assessment)

    # Step 3: Generate Treatment Recommendations
    # -------------------------------------------
    # ✅ Generate the treatment plan using MRI findings and the risk assessment
    raw_treatment_plan = generate_treatment_plan(mri_findings, risk_assessment, fine_tuned_generator)
    treatment_plan = extract_output_section(raw_treatment_plan)

    # Step 4: Combine All Sections into a Final Report
    # -------------------------------------------------
    # ✅ Format the content neatly with headers and spacing
    report = (
        "================ MRI Diagnostic Report ================\n\n"
        "**MRI Findings:**\n"
        f"{mri_findings}\n\n"
        "**Risk Assessment:**\n"
        f"{risk_assessment}\n\n"
        "**Treatment Recommendations:**\n"
        f"{treatment_plan}\n"
        "========================================================"
    )

    # Step 5: Return the Final Report
    return report

# Sample Usage
report = generate_mri_diagnostic_report(mri_description, fine_tuned_generator)
print(report)


**MRI Findings:**
    Tumor Classification: Glioma (likely astrocytic or oligodendroglial in origin)
    Location: Right frontal lobe
    Size/Extent: 4.5 cm
    Type: Hyperintense on T2-weighted imaging, with irregular margins and heterogeneous
          enhancement following contrast administration
    Distribution Pattern: Intracranial mass with irregular margins, causing significant
                        midline shift and compression of the right lateral ventricle
    Mass Effect: Significant vasogenic edema present, causing midline shift of
                approximately 5 mm and partial effacement of the sulci 

**Risk Assessment:**
    Based on the MRI findings, the risk of malignancy is estimated to be [High (70/100)].
    The contribution to the risk score is as follows:
      - Tumor Classification: Contributes 20% to the total risk score. The glioma
        classification is consistent with a medium to high-grade tumor, which accounts for
        [80/100] of the total risk 